In [12]:
import requests
from bs4 import BeautifulSoup
import re
import os

In [13]:
BASE_URL = "https://docs.rs"
CRATE_URL = "https://docs.rs/burn/latest/burn/"

In [31]:
def to_markdown(html_content):
    """
    Convertit rapidement (et de façon rudimentaire) un contenu HTML en Markdown.
    Pour quelque chose de plus élaboré, on peut envisager l'utilisation
    de bibliothèques comme html2text, ou l’écriture de règles plus fines.
    """
    # On peut faire qq conversions de base, par exemple remplacer <h1>, <h2>... 
    # et enlever les balises de code, etc.
    # Ici, on fait quelque chose de très minimaliste :
    text = re.sub(r'<h1[^>]*>(.*?)</h1>', r'# \1', html_content, flags=re.DOTALL)
    text = re.sub(r'<h2[^>]*>(.*?)</h2>', r'## \1', text, flags=re.DOTALL)
    text = re.sub(r'<h3[^>]*>(.*?)</h3>', r'### \1', text, flags=re.DOTALL)
    text = re.sub(r'<code[^>]*>(.*?)</code>', r'`\1`', text, flags=re.DOTALL)
    # Supprimer toutes les balises HTML restantes
    text = re.sub(r'<[^>]+>', '', text)
    # Retirer les espaces multiples
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = text.strip()
    return text

In [32]:
def get_soup(url):
    """Retourne le BeautifulSoup d'une URL."""
    r = requests.get(url)
    r.raise_for_status()
    return BeautifulSoup(r.text, 'html.parser')

In [37]:
def extract_main_content(soup):
    main_div = soup.find(id="main")
    if not main_div:
        main_div = soup.body.find(id="main-content")

    # Supprimer la navigation, barres latérales ou tout autre élément non-essentiel
    for nav_tag in main_div.find_all(["nav", "aside", "rustdoc-search", "copy-path", "rustdoc-toolbar"]):
        nav_tag.decompose()

    # Par exemple, supprimer des disclaimers ou sections non-requises
    disclaimers = main_div.find_all(class_="disclaimer")
    for d in disclaimers:
        d.decompose()

    return str(main_div)

In [38]:
def get_doc_links(soup):
    """
    Récupère les liens internes importants (modules, traits, macros, structs, etc.).
    Ils apparaissent généralement dans la <section class='sidebar-elems'> 
    ou bien dans des <a> avec un href commençant par './' ou similaire.
    """
    doc_links = set()
    
    # Exemples de selecteurs possibles. 
    # On pourrait aussi parcourir toutes les balises <a> et filtrer.
    sidebar = soup.find("section", class_="sidebar-elems")
    if sidebar:
        for a in sidebar.find_all("a", href=True):
            href = a["href"]
            # On prend les liens internes à la doc (souvent "./" ou "../trait...")
            if href.startswith("./") or href.startswith("../"):
                doc_links.add(href)
    
    # Conversion en liens absolus
    absolute_links = []
    for link in doc_links:
        if link.startswith("./"):
            absolute_links.append(CRATE_URL + link[2:])  # retire "./" et ajoute base
        elif link.startswith("../"):
            # remontée d'un niveau : on retire '../' et on concatène
            # Cependant, il peut y avoir plusieurs niveaux. À ajuster si besoin.
            # Ex : "../foo" => ndarray/foo
            link_part = link.lstrip("./")
            absolute_links.append(os.path.join(CRATE_URL, link_part))
        else:
            # Autres cas, potentiellement des liens complets
            if link.startswith("http"):
                absolute_links.append(link)
            else:
                absolute_links.append(CRATE_URL + link)
                
    return list(set(absolute_links))

In [39]:
def scrape_ndarray_docs():
    # 1. Récupérer la page d'accueil
    main_soup = get_soup(CRATE_URL)
    
    # 2. Récupérer le contenu principal
    main_content_html = extract_main_content(main_soup)
    
    # 3. Conversion en Markdown
    main_content_md = to_markdown(main_content_html)
    
    # 4. Récupérer la liste des liens internes
    links = get_doc_links(main_soup)
    print(links)
    
    all_docs_md = main_content_md + "\n\n"
    
    visited = set()  # pour éviter de scraper plusieurs fois la même page
    for link in links:
        if link not in visited:
            visited.add(link)
            try:
                page_soup = get_soup(link)
                page_content_html = extract_main_content(page_soup)
                page_content_md = to_markdown(page_content_html)
                
                all_docs_md += f"## Page : {link}\n\n"
                all_docs_md += page_content_md + "\n\n"
                
            except Exception as e:
                print(f"Impossible de scraper {link} : {e}")
    
    # 6. Sauvegarder le tout dans un fichier Markdown
    with open("ndarray_doc.md", "w", encoding="utf-8") as f:
        f.write(all_docs_md)


In [40]:
scrape_ndarray_docs()

[]
